In [1]:
import numpy as np
import pandas as pd
#---------------------csv location------------------------------------------------
dataLocation = r'/Users/yuliang/Desktop/change_data/data_pro.csv'
data_normal = pd.read_csv(dataLocation)
data_normal_used = data_normal[['oindex','match','team','player','won','tsync','tier','zone']]
#data_test = data_normal_used.drop_duplicates('match')
#print(data_test)
#----------------------time process------------------------------------------
df_time = data_normal_used.groupby(['match'])['tsync'].max()
newdf_time = pd.DataFrame({'match_end' : df_time}).reset_index()
newdf_time2 = newdf_time.sort(['match'], ascending = [True])
match_end = newdf_time2[['match_end']]
#--------------------initial state-------------------------------------------
data_initial_1 = data_normal[data_normal['tsync']==0]
#--------------------initial state won---------------------------------------
data_initial_won1 = data_initial_1[data_initial_1['won']==1]
data_initial_won2 = data_initial_won1.sort(['match'], ascending = [True]).reset_index()
data_initial_won = data_initial_won2[['index','oindex','match','team','player','won','tier','zone']]
data_initial_wondr = data_initial_won.drop_duplicates('match')
#--------------------initial state lose--------------------------------------
data_initial_lose1 = data_initial_1[data_initial_1['won']==0]
data_initial_lose2 = data_initial_lose1.sort(['match'], ascending = [True]).reset_index()
data_initial_lose = data_initial_lose2[['index','oindex','match','team','player','won','tier','zone']]
data_initial_losedr = data_initial_lose.drop_duplicates('match')
#---------------------zone change statistic---------------------------------------------------------------
def zone_change (df, player_index, real_index, player_name, zone, consistant_count):
    i = player_index #new player index, start from 0
    j = 0
    initial_player = df.get_value(real_index, player_name)
    initial_state = df.get_value(real_index, zone)
    consistant_count[i][j] = 0 
    
    for index in range(real_index, len(df.index)):
        current_player = df.get_value(index, player_name)
        current_state = df.get_value(index, zone)
        
        if current_player != initial_player:
            break
        else: 
            if current_state == initial_state:
                consistant_count[i][j] = consistant_count[i][j] + 1
            else:
                j = j + 1
                consistant_count[i][j] = 1
                initial_state = current_state
    return consistant_count
            
array1 = [[0 for x in range(500)] for x in range(len(data_initial_won.index))]
array2 = [[0 for x in range(500)] for x in range(len(data_initial_won.index))]

for index in range(len(data_initial_won.index)):
    real_index = data_initial_won.get_value(index, 'index')
    array2 = zone_change(data_normal_used, index, real_index,'player','zone', array1)

print("count begins")

match_count = [0 for x in range(len(data_initial_wondr.index))]
for index in range(len(data_initial_wondr.index)):
    new_index = index*5
    new_index_stop = new_index + 5
    for i in range(new_index, new_index_stop):
        for j in range(500):
            if array2[i][j] == 0:
                break
            elif array2[i][j] >= 5:
                match_count[index] = match_count[index] + 1 

change_zone_count = pd.Series(match_count)

df1 = pd.DataFrame(change_zone_count, columns=['change_zone_count'])
df2 = data_initial_wondr[['match','team','won','tier']].reset_index()
df3 = df2[['match','team','won','tier']]
newdf_time = pd.DataFrame({'match_end' : df_time}).reset_index()
df = pd.concat([df3, df1, match_end], join='outer', axis=1)

df4 = df['change_zone_count']*60/(5*df['match_end'])
df5 = pd.DataFrame({'change_per_min_person': df4})
df_final = pd.concat([df, df5], join='outer', axis=1)
#print(df_final)
#newdf_time = pd.DataFrame({'match_end' : df_time}).reset_index()
#print(df5)

#df_test['change_zone_count'] = df_test.apply(lambda row: (row['change_zone_count']/row['match_end']),axis=1)

df_sort = df_final.sort(['change_per_min_person'], ascending=[True]).reset_index()
df_sort_final = df_sort[['match','team','won','tier','change_per_min_person']]
#print(df_sort_final)
#mean_change = df['change_zone_count'].mean()
#std_change = df['change_zone_count'].std()

df_sort_final.to_csv('/Users/yuliang/Desktop/pro_change_won.csv', index=False, sep=',' )
print("succeed")
#df_test.to_csv('/Users/yuliang/Desktop/normal_change_test.csv', index=False, sep=',' )
#print("succeed")
#print(df_sort_final)
#print("mean change is %.4f" % mean_change)
#print("std is %.4f" % std_change)













count begins
succeed
